## 8.1 컨볼루션

### 8.1.1 컨볼루션의 역할
- 이산 컨볼루션(discrete convolution) : 2차원 이미지에 가중치 행렬을 스칼라곱을 수행하는 것으로 정의
    * 가중치 행렬 = 커널
  
[특징]  
- 지역성 : 이동된 커널 * 이미지 스칼라곱 
- 평행이동 불변성 : 이미지 전 영역에 대해 동일한 커널 가중치
- 적은 파라미터 : FC와는 달리 컨볼루션에서의 파라미터 수는 이미지 픽셀 수에 의존하지 않음  
    -> 컨볼루션 커널 크기와 모델에서 얼마나 많은 컨볼루션 필터(출력 채널 수)를 쓰는지에 의존
    
## 8.2 컨볼루션 사용해보기
- nn.Conv1d : 시계열용
- nn.Conv2d : 이미지용
- nn.Conv3d : 용적 데이터나 동영상용

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [3]:
from torchvision import datasets, transforms
data_path = r'C:\Users\knuyh\Desktop\민지\스터디\파이토치 딥러닝 마스터\Minji\Part01\CH07\Data/'

cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [4]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

In [5]:
connected_model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.Tanh(),
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 2))

In [6]:
first_model = nn.Sequential(
                nn.Linear(3072, 512),
                nn.Tanh(),
                nn.Linear(512, 2),
                nn.LogSoftmax(dim=1))

In [7]:
conv = nn.Conv2d(3, 16, kernel_size = 3) # 입력 RGB 3, 출력 피처 16, kernel_size=(3, 3)
conv

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))

In [8]:
conv.weight.shape, conv.bias.shape # [출력, 입력, 커널, 커널]

(torch.Size([16, 3, 3, 3]), torch.Size([16]))

In [9]:
img, _ = cifar2[0]
output = conv(img.unsqueeze(0))
img.unsqueeze(0).shape, output.shape  # B X C X H X W, 출력의 픽셀 30으로 잘림

(torch.Size([1, 3, 32, 32]), torch.Size([1, 16, 30, 30]))

### 8.2.1 경계 패딩하기
- 패딩 여부 상관없이 weight, bias 크기는 변하지 않음
- 컨볼루션과 이미지 크기 변경 문제를 별도로 분리해 기억해야 하는 것을 하나 줄이는데 도움
- 컨볼루션 구조 자체에 더 신경쓸 수 있다.

In [10]:
conv = nn.Conv2d(3, 1, kernel_size=3, padding=1)
output = conv(img.unsqueeze(0))
img.unsqueeze(0).shape, output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 1, 32, 32]))

In [11]:
conv.weight.shape, conv.bias.shape

(torch.Size([1, 3, 3, 3]), torch.Size([1]))

### 8.2.2 컨볼루션으로 피처 찾아내기
- CH07에서는 weight, bias 랜덤하게 초기화하고, 역전파를 통해 학습되는 파라미터
- 컨볼루션은 직접 가중치 설정 가능

In [12]:
# bias 0으로 제거해 교란 변수 배제
with torch.no_grad() :
    conv.bias.zero_()
    
# 가중치에 상수값 넣어 출력에서의 각 픽셀이 자신의 이웃 픽셀에 대한 평균 가지게
with torch.no_grad() :
    conv.weight.fill_(1.0 / 9.0) # 3 X 3 이웃

In [13]:
conv.bias, conv.weight

(Parameter containing:
 tensor([0.], requires_grad=True),
 Parameter containing:
 tensor([[[[0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111]],
 
          [[0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111]],
 
          [[0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111],
           [0.1111, 0.1111, 0.1111]]]], requires_grad=True))

In [14]:
output = conv(img.unsqueeze(0))
# 흐려진 이미지 : 각 출력의 픽셀은 자신의 주변 픽셀에 대한 평균이기에 출력 픽셀에서 이러한 상관관계 반영해 픽셀 간의 변화가 부드러워짐

In [15]:
conv = nn.Conv2d(3, 1, kernel_size=3, padding=1)

with torch.no_grad():
    conv.weight[:] = torch.tensor([[-1.0, 0.0, 1.0],
                                   [-1.0, 0.0, 1.0],
                                   [-1.0, 0.0, 1.0]])
    conv.bias.zero_()
    
# 가로로 인접한 두 영역 사이의 수직 경계 탐색

### 8.2.3 깊이와 풀링으로 한 단계 더 인식하기
- 큰 이미지에서 작은 이미지로 다운샘플링  
 : 컨볼루션을 차례로 층층이 쌓으며 동시에 연속적인 컨볼루션 사이의 이미지 다운샘플링
     * 네 개의 픽셀 평균하기 : 평균 풀링
     * 네 개의 픽셀 중 최댓값 : 맥스 풀링 ; 데이터의 3/4 버림
     * stride 하며 컨볼루션 수행하되, n번째 픽셀만 계산

In [16]:
pool = nn.MaxPool2d(2) # 이미지 절반으로 줄임
output = pool(img.unsqueeze(0))
img.unsqueeze(0).shape, output.shape

(torch.Size([1, 3, 32, 32]), torch.Size([1, 3, 16, 16]))

### 8.2.4 우리의 신경망에 적용하기

In [17]:
model = nn.Sequential(
        nn.Conv2d(3, 16, kernel_size=3, padding=1),
        nn.Tanh(),
        nn.MaxPool2d(2),
        nn.Conv2d(16, 8, kernel_size=3, padding=1),
        nn.Tanh(),
        nn.MaxPool2d(2),
        nn.Linear(8 * 8 * 8, 32),  # FC
        nn.Tanh(),
        nn.Linear(32, 2))
## nn.Sequential() : 각 모듈의 출력을 명시적으로 볼 수 없다.

In [18]:
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(18090, [432, 16, 1152, 8, 16384, 32, 64, 2])

In [32]:
model(img.unsqueeze(0))
# 

RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x8 and 512x32)